<a href="https://colab.research.google.com/github/jlembury/kickflip-cartography-demo/blob/main/kickflip_cartography_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="margin-left: 0; margin-right: auto;">
    <tr>
        <td><img src="https://github.com/jlembury/kickflip-cartography-demo/blob/main/img/img-nada-kJvjDlqTkR0-unsplash.jpg?raw=1"></td>
        <td><h1 style="margin-bottom: 0;">Kickflip Cartography: </h1>
            <h3 style="margin-top: 0;">Finding the Perfect Location for a Skate Park</h3><br><br>
            <h4 style="margin-bottom: 0;">GIS Awareness Week 2025, hosted by AIRC x Ecologik</h4>
            <h4 style="margin-top: 0; margin-bottom: 0;">Jessica Embury <em>(she/her/hers)</em></h4>
            <h4 style="margin-top: 0; margin-bottom: 0;">Email: <a href="mailto:jessica@gisjess.com">jessica@gisjess.com</a></h4>
        </td>
    </tr>
</table>
<hr>
<br>

### Project Overview
A local community group wants to build a skate park somewhere in the City of San Diego. The group wants to create a safe hangout for skateboarders <em>and</em> maximize access to skate parks throughout the city. To meet these objectives, the site for the future skate park must:
1. be located in a <b>San Diego public park</b> at least 1 mile from an <b>existing skate park</b>, and
2. be located in a <b>zip code</b> with demonstrated need, per <b>2025 San Diego police call records</b>.
<br>
<hr>
<br>

### Data Sources
- <em>parks</em>: San Diego Regional Data Warehouse, https://tinyurl.com/sd-data-warehouse
- <em>skateParks</em>: The City of San Diego, https://www.sandiego.gov/park-and-recreation/centers/skateparks
- <em>zipcodes</em>: San Diego Regional Data Warehouse, https://tinyurl.com/sd-data-warehouse
- <em>policeCalls</em>: Data SD, https://data.sandiego.gov/datasets/police-calls-for-service/
<br>
<hr>
<br>

### Spatial Operations
<img src="https://github.com/jlembury/kickflip-cartography-demo/blob/main/img/img-spatial-operations.jpg?raw=1">
<hr>

###Demo Set Up

In [ ]:
# Install libraries
!pip install geopandas
!pip install mapclassify

In [ ]:
# Import libraries
import pandas as pd
import geopandas as gpd
import time
import numpy as np
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import folium
import mapclassify

In [ ]:
# Assign input data URLs to variables
urlBase = "https://raw.githubusercontent.com/jlembury/kickflip-cartography-demo/refs/heads/main/data"
skateParksFile = f"{urlBase}/skate-park-locations.csv"
parksFile = f"{urlBase}/Parks_SD.geojson"
policeCallsFile = f"{urlBase}/skateboarding-police-calls-2025.csv"
zipcodesFile = f"{urlBase}/Zipcodes.geojson"

# Data outputs (pre-saved)
skateParkPointsFile = f"{urlBase}/skateParkPoints.geojson"
skateParkBuffersFile = f"{urlBase}/skateParkBuffers.geojson"
parkCandidatesFile = f"{urlBase}/parkCandidates.geojson"
policeCallPointsFile = f"{urlBase}/policeCallPoints.geojson"
zipcodeCountsFile = f"{urlBase}/zipcodeCountsFile.geojson"
parkFinalistsFile = f"{urlBase}/parkFinalists.geojson"

<hr>

###Objective 1
Identify San Diego parks that are more than 1 mile from a skate park.

Step 1. <u>Geocode</u> the street addresses in <em>skateParks</em> → <em>skateParkPoints</em>

In [ ]:
# Load skateParks data
skateParks = pd.read_csv(skateParksFile)
print(f"There are {len(skateParks)} skate parks in San Diego.")
skateParks.head(2)

In [ ]:
# Create geolocator to interact with the geocoding service, Nominatim.
geolocator = Nominatim(user_agent="kickflip-cartography-demo")

In [ ]:
# Geocode the address text to get latitude and longitude coordinates.
'''
geometry = []  # Empty list object to store mappable point geometry
for i,row in skateParks.iterrows():  # For each skate park
  location = geolocator.geocode(skateParks.at[i, "address"]) # Geocode the address text
  geometry.append(Point(location.longitude, location.latitude))  # Create a point and store it in the geometry list
  time.sleep(1)  # Wait 1 second, per terms of service

# Merge skate park data and skate park geometry
skateParkPoints = gpd.GeoDataFrame(skateParks, geometry=geometry, crs="EPSG:4326")
'''
skateParkPoints = gpd.read_file(skateParkPointsFile)

skateParkPoints.to_crs("EPSG:3857", inplace=True)  # select projection
skateParkPoints.head(2)

In [ ]:
# map locations of skate parks
m1 = folium.Map(location=[32.75, -117.2], zoom_start=10, width=800, height=600)
folium.GeoJson(
    skateParkPoints,
    popup=folium.GeoJsonPopup(fields=["skate_park_name", "address"],aliases=["Skate Park Name", "Address"]),
    marker=folium.Marker(icon=folium.Icon(color="blue", icon="dot-circle", prefix="fa"))
).add_to(m1)  # Add skate parks to the map
m1

Step 2. Use a <u>buffer</u> operation to create 1-mile zones around the points in <em>skateParkPoints</em>  → <em>skateParkBuffers</em>

In [ ]:
# Prepare data for buffers
skateParkBuffers = skateParkPoints[["skate_park_name", "address"]]

# create geometry for 1-mile buffers (1,610 meters)
skateParkBuffers["geometry"] = skateParkPoints.geometry.buffer(1610)
skateParkBuffers = skateParkBuffers.set_geometry("geometry")
skateParkBuffers.head(2)

In [ ]:
# map locations of skate parks and their 1-mile buffers
m2 = folium.Map(location=[32.75, -117.15], zoom_start=12, width=800, height=600)
folium.GeoJson(
    skateParkBuffers
).add_to(m2)  # Add buffers to the map
folium.GeoJson(
    skateParkPoints,
    popup=folium.GeoJsonPopup(fields=["skate_park_name", "address"],aliases=["Skate Park Name", "Address"]),
    marker=folium.Marker(icon=folium.Icon(color="blue", icon="star", prefix="fa"))
).add_to(m2)  # Add skate parks to the map
m2

Step 3. Use a <u>spatial join</u> to remove <em>parks</em> that overlap with <em>skateParkBuffers</em> → <em>parkCandidates</em>

In [ ]:
# Load parks data
parks = gpd.read_file(parksFile)[["full_name", "address_lo", "community", "geometry"]].rename(columns={"full_name":"park_name", "address_lo":"address"})
parks.to_crs("EPSG:3857", inplace=True)  # set projection

print(f"There are {len(parks)} parks in San Diego.")
parks.head(2)


In [ ]:
# Use a spatial join to merge data when a park is within a 1-mile skate park buffer.
parkCandidates = parks.sjoin(skateParkBuffers, how="left")

# Save parks data for parks that did NOT join with a skate park (more than 1 mile away)
parkCandidates = parkCandidates[parkCandidates['skate_park_name'].isna()][["park_name", "address_left", "community", "geometry"]].rename(columns={"address_left":"address"})

print(f"There are {len(parkCandidates)} San Diego parks that are more than 1 mile from a skate park.")
parkCandidates.head(2)

In [ ]:
# map locations of skate parks, 1-mile buffers, and park candidates
m3 = folium.Map(location=[32.75, -117.15], zoom_start=12, width=800, height=600)
folium.GeoJson(
    parkCandidates,
    popup=folium.GeoJsonPopup(fields=["park_name", "address"],aliases=["Park Name", "Address"]),
    color="green"
).add_to(m3)  # Add park candidates to the map
folium.GeoJson(
    skateParkBuffers
).add_to(m3)  # Add buffers to the map
folium.GeoJson(
    skateParkPoints,
    popup=folium.GeoJsonPopup(fields=["skate_park_name", "address"],aliases=["Skate Park Name", "Address"]),
    marker=folium.Marker(icon=folium.Icon(color="blue", icon="star", prefix="fa"))
).add_to(m3)  # Add skate parks to the map
m3

<hr>

###Objective 2
Find park(s) located in the zipcode with the most skateboarding police calls.

Step 4. <u>Geocode</u> the skateboarding incident locations from <em>policeCalls</em> → <em>policeCallPoints</em>

In [ ]:
# Load policeCalls data.
policeCalls = pd.read_csv(policeCallsFile)
print(f"In 2025, there were {len(policeCalls)} police incidents about skateboarding.")
policeCalls.head(2)

In [ ]:
'''
# Geocode the address text to get latitude and longitude coordinates.
geometry = [] # Empty list object to store mappable point geometry
for i,row in policeCalls.iterrows():  # For each incident
  location = geolocator.geocode(policeCalls.at[i, "address"])  # Geocode the address text
  geometry.append(Point(location.longitude, location.latitude))  # Create a point and store it in the geometry list
  time.sleep(1)  # Wait 1 second, per terms of service

# Merge police call data and police call geometry
policeCallPoints = gpd.GeoDataFrame(policeCalls, geometry=geometry, crs="EPSG:4326")
'''

policeCallPoints = gpd.read_file(policeCallPointsFile)

policeCallPoints.to_crs("EPSG:3857", inplace=True)  # select projection
policeCallPoints.head(2)

In [ ]:
# map locations of 2025 police incidents
m4 = folium.Map(location=[32.75, -117.15], zoom_start=12, width=800, height=600)
folium.GeoJson(
    policeCallPoints,
    popup=folium.GeoJsonPopup(fields=["incident-number", "address"],aliases=["Incident Number", "Address"]),
    marker=folium.Marker(icon=folium.Icon(color="red", icon="warning", prefix="fa"))
).add_to(m4)  # Add incident locations to the map
m4

Step 5. Use a <u>spatial join</u> to count the number of incidents per zipcode in <em>zipcodes</em> → <em>zipcodeCounts</em>

In [ ]:
# Load zipcodes data.
zipcodes = gpd.read_file(zipcodesFile).query("COMMUNITY == 'San Diego'")[["ZIP", "geometry"]].rename(columns={"ZIP":"zipcode"})
zipcodes["zipcode"] = zipcodes["zipcode"].astype(str)
zipcodes.to_crs("EPSG:3857", inplace=True)  # select projection.
print(f"There are {len(zipcodes)} zipcodes in San Diego.")
zipcodes.head(2)

In [ ]:
# Use a spatial join to merge data when a police incident occurred within a zipcode.
zipcodeCounts = zipcodes.sjoin(policeCallPoints, how="left")

# Find the zipcode with the highest number of police incidents.
zipcodeCounts["index_right"] = zipcodeCounts["index_right"].notna()
zipcodeCounts["count"] = np.where(zipcodeCounts["index_right"] == False, 0, 1)
zipcodeCounts = zipcodes[["zipcode", "geometry"]].merge(zipcodeCounts[["zipcode", "count"]].groupby("zipcode").sum("count"), on="zipcode", how="left").sort_values(["count", "zipcode"], ascending=False)
zipcodeCounts.to_crs("EPSG:3857", inplace=True) # select projection
zipcodeCounts.head(1)

In [ ]:
# map locations of 2025 police incidents with zipcode boundaries
m5a = folium.Map(location=[32.75, -117.15], zoom_start=12, width=800, height=600)
folium.GeoJson(
    zipcodes,
    popup=folium.GeoJsonPopup(fields=["zipcode"],aliases=["Zipcode"]),
    color="black",
    weight=1,
    fill=False
).add_to(m5a)  # Add zipcode boundaries to the map
folium.GeoJson(
    policeCallPoints,
    popup=folium.GeoJsonPopup(fields=["incident-number", "address"],aliases=["Incident Number", "Address"]),
    marker=folium.Marker(icon=folium.Icon(color="red", icon="warning", prefix="fa"))
).add_to(m5a)  # Add incident locations to the map
m5a

In [ ]:
# map number of 2025 police incidents per zipcode using choropleth symbology
m5b = folium.Map(location=[32.75, -117.15], zoom_start=12, width=800, height=600)
choropleth=folium.Choropleth(
    geo_data=zipcodeCounts,
    data=zipcodeCounts,
    columns=["zipcode", "count"],
    key_on="feature.properties.zipcode",
    fill_color="YlOrRd",
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name="Number of 2025 Police Incidents",
).add_to(m5b)  # Add shaded zipcodes to the map
folium.features.GeoJsonPopup(
    fields=["zipcode", "count"], # Fields from geojson properties and joined data
    aliases=["Zipcode", "Incident Count"]
).add_to(choropleth.geojson)  # Create zipcode popups
m5b

Step 6. Use a <u>spatial join</u> to find park(s) (<em>parkCandidates</em>) in the zipcode(s) (<em>zipcodeCounts</em>) with the most skateboarding incidents → <em>parkFinalists</em>

In [ ]:
# Identify parks that are more than 1 mile away (parkCandidates) and are within the max-incident zipcode.
maxCountZipcode = zipcodeCounts.loc[zipcodeCounts['count'].idxmax(), "zipcode"]  # get max-incident zipcode
parkFinalists = parkCandidates.sjoin(zipcodeCounts.query("zipcode == @maxCountZipcode"), how="left").dropna(subset=["index_right"])[["park_name", "address", "community", "geometry"]]
parkFinalists

In [ ]:
# map number of 2025 police incidents per zipcode using choropleth symbology
m6 = folium.Map(location=[32.74, -117.13], zoom_start=13, width=800, height=600)
choropleth2=folium.Choropleth(
    geo_data=zipcodeCounts,
    data=zipcodeCounts,
    columns=["zipcode", "count"],
    key_on="feature.properties.zipcode",
    fill_color="YlOrRd",
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name="Number of 2025 Police Incidents",
).add_to(m6)  #  Add shaded zipcodes to the map
folium.features.GeoJsonPopup(
    fields=["zipcode", "count"], # Fields from geojson properties and joined data
    aliases=["Zipcode", "Incident Count"]
).add_to(choropleth2.geojson)  #  Create zipcode popups
folium.GeoJson(
    skateParkBuffers
).add_to(m6)  #  Add buffers to the map
folium.GeoJson(
    parkFinalists,
    popup=folium.GeoJsonPopup(fields=["park_name", "address"],aliases=["Park Name", "Address"]),
    color="green"
).add_to(m6)  # Add park finalists to the map
folium.GeoJson(
    skateParkPoints,
    popup=folium.GeoJsonPopup(fields=["skate_park_name", "address"],aliases=["Skate Park Name", "Address"]),
    marker=folium.Marker(icon=folium.Icon(color="blue", icon="dot-circle", prefix="fa"))
).add_to(m6)  # Add skate parks to the map
folium.GeoJson(
    gpd.GeoDataFrame(parkFinalists[["park_name", "address"]], geometry=parkFinalists.geometry.centroid),
    popup=folium.GeoJsonPopup(fields=["park_name", "address"],aliases=["Park Name", "Address"]),
    marker=folium.Marker(icon=folium.Icon(color="green", icon="star", prefix="fa"))
).add_to(m6)  # Add points for the park finalists to the map
m6